In [ ]:
!pip install xlsxwriter pydrive

In [2]:
#%% import
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import json
import re


#%% functions

def get_posts_from_feed(feed_url, start_index=1, max_results=500):
    """
    Pobiera posty z Blogger Atom feed
    Blogger feed: /feeds/posts/default?start-index=X&max-results=Y
    """
    posts = []
    try:
        # Dodaj parametry do URL
        url = f"{feed_url}?start-index={start_index}&max-results={max_results}&alt=rss"

        r = requests.get(url)
        r.encoding = 'utf-8'
        soup = BeautifulSoup(r.text, 'xml')

        # W Atom feed linki są w <link> z rel="alternate"
        for item in soup.find_all('item'):
            link_elem = item.find('link')
            if link_elem:
                link = link_elem.text.strip()
                if link and 'blogspot.com' in link:
                    posts.append(link)

        # Alternatywnie próbuj <entry> (Atom format)
        if not posts:
            for entry in soup.find_all('entry'):
                for link in entry.find_all('link', rel='alternate'):
                    href = link.get('href')
                    if href and 'blogspot.com' in href:
                        posts.append(href)

        return posts
    except Exception as e:
        print(f"Błąd pobierania feed: {e}")
        return []


def get_all_posts(blog_url):
    """
    Pobiera wszystkie posty z bloga Blogger używając feed
    """
    feed_url = blog_url.rstrip('/') + '/feeds/posts/default'

    print(f"Używam feed: {feed_url}")

    all_posts = []
    start_index = 1
    batch_size = 500  # Maksymalny rozmiar batcha dla Blogger

    print("\nPobieranie postów z feed...")

    while True:
        print(f"  Pobieranie od pozycji {start_index}...")
        posts = get_posts_from_feed(feed_url, start_index, batch_size)

        if not posts:
            print(f"  Brak więcej postów")
            break

        all_posts.extend(posts)
        print(f"  Pobrano {len(posts)} postów")

        # Jeśli otrzymaliśmy mniej niż batch_size, to koniec
        if len(posts) < batch_size:
            break

        start_index += batch_size
        time.sleep(0.5)  # Ostrożność

    # Usuń duplikaty
    all_posts = list(set(all_posts))

    return all_posts


def filter_article_links(all_links):
    """
    Filtruje linki - zostawia tylko artykuły
    """
    article_links = []

    for link in all_links:
        # Wykluczamy strony statyczne i inne nie-posty
        # Blogger posty zwykle mają /YYYY/MM/ w URL lub /.html

        # Wykluczamy:
        if '/search/' in link:  # Wyszukiwanie
            continue
        if '/p/' in link:  # Statyczne strony
            continue
        if link.endswith('.com/') or link.endswith('.com'):  # Główna strona
            continue

        # Akceptujemy posty
        article_links.append(link)

    return article_links


#%% main execution

if __name__ == "__main__":
    blog_url = "https://jacekglomb.blogspot.com/"

    print("="*60)
    print("Pobieranie linków z jacekglomb.blogspot.com")
    print("="*60)

    # Pobierz wszystkie posty
    all_posts = get_all_posts(blog_url)

    print(f"\nZnaleziono {len(all_posts)} postów z feed")

    # Filtruj
    article_links = filter_article_links(all_posts)

    print(f"Po filtrowaniu: {len(article_links)} artykułów")

    if not article_links:
        print("\n⚠️  Nie znaleziono artykułów!")
        print("Sprawdź ręcznie feed:")
        print(f"  {blog_url}feeds/posts/default")
        exit(1)

    # Pokaż przykłady
    if article_links:
        print("\nPrzykładowe linki (pierwsze 5):")
        for i, link in enumerate(article_links[:5], 1):
            print(f"  {i}. {link}")

    # Sortuj
    article_links.sort()

    # Zapisz do pliku tekstowego
    with open('glomb_linki.txt', 'w', encoding='utf-8') as f:
        for link in article_links:
            f.write(link + '\n')

    # Zapisz do JSON
    output_data = {
        'source': blog_url,
        'feed': blog_url + 'feeds/posts/default',
        'total_links': len(article_links),
        'links': article_links,
        'errors': []
    }

    with open('glomb_linki.json', 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=2)

    # Raport
    print("\n" + "="*60)
    print("RAPORT")
    print("="*60)
    print(f"Znaleziono artykułów: {len(article_links)}")
    print(f"\nZapisano do:")
    print(f"  - glomb_linki.txt")
    print(f"  - glomb_linki.json")
    print("="*60)

Pobieranie linków z jacekglomb.blogspot.com
Używam feed: https://jacekglomb.blogspot.com/feeds/posts/default

Pobieranie postów z feed...
  Pobieranie od pozycji 1...
  Pobrano 149 postów

Znaleziono 149 postów z feed
Po filtrowaniu: 149 artykułów

Przykładowe linki (pierwsze 5):
  1. https://jacekglomb.blogspot.com/2013/06/jak-zostaem-kibicem-arsenalu-czesc-ii.html
  2. https://jacekglomb.blogspot.com/2012/06/czesi-wola-hokej.html
  3. https://jacekglomb.blogspot.com/2011/09/orkiestra.html
  4. https://jacekglomb.blogspot.com/2014/02/do-marka-fiedora-sow-kilka_18.html
  5. https://jacekglomb.blogspot.com/2012/10/zapach-zuzla.html

RAPORT
Znaleziono artykułów: 149

Zapisano do:
  - glomb_linki.txt
  - glomb_linki.json


In [3]:
#%% import
from __future__ import unicode_literals
import re
import time
from datetime import datetime
from time import mktime
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import json
import xlsxwriter


#%% functions

def date_change_format(date_string):
    """
    Konwertuje datę z różnych formatów na "YYYY-MM-DD"
    """
    try:
        date_string = ' '.join(date_string.strip().split())

        if re.match(r'\d{4}-\d{2}-\d{2}', date_string):
            return date_string[:10]

        if 'T' in date_string:
            return date_string.split('T')[0]

        lookup_table = {
            "stycznia": "01", "lutego": "02", "marca": "03", "kwietnia": "04",
            "maja": "05", "czerwca": "06", "lipca": "07", "sierpnia": "08",
            "września": "09", "października": "10", "listopada": "11", "grudnia": "12",
            "styczeń": "01", "luty": "02", "marzec": "03", "kwiecień": "04",
            "maj": "05", "czerwiec": "06", "lipiec": "07", "sierpień": "08",
            "wrzesień": "09", "październik": "10", "listopad": "11", "grudzień": "12"
        }

        for k, v in lookup_table.items():
            date_string = date_string.replace(k, v)

        if re.match(r'\d{1,2}\.\d{1,2}\.\d{4}', date_string):
            result = time.strptime(date_string, "%d.%m.%Y")
        else:
            result = time.strptime(date_string, "%d %m %Y")

        changed_date = datetime.fromtimestamp(mktime(result))
        return format(changed_date.date())
    except Exception as e:
        print(f"Błąd konwersji daty '{date_string}': {e}")
        return "no date"


def dictionary_of_article(article_link):
    """
    Pobiera szczegóły artykułu z jacekglomb.blogspot.com (Blogger)
    """
    try:
        r = requests.get(article_link)
        r.encoding = 'utf-8'
        html_text = r.text

        while '429 Too Many Requests' in html_text:
            time.sleep(5)
            r = requests.get(article_link)
            r.encoding = 'utf-8'
            html_text = r.text

        soup = BeautifulSoup(html_text, 'lxml')

        # Data publikacji - Blogger używa różnych struktur
        try:
            # Opcja 1: <time> lub <abbr class="published">
            date_element = soup.find('time')
            if not date_element:
                date_element = soup.find('abbr', class_='published')
            if not date_element:
                date_element = soup.find('span', class_=lambda x: x and 'date' in str(x).lower())

            if date_element:
                date_text = date_element.get('datetime') or date_element.get('title') or date_element.get_text(strip=True)
                date_of_publication = date_change_format(date_text)
            else:
                date_of_publication = "no date"
        except Exception as e:
            print(f"Błąd parsowania daty dla {article_link}: {e}")
            date_of_publication = "no date"

        # Tytuł - Blogger
        try:
            title_element = soup.find('h1', class_=lambda x: x and 'title' in str(x).lower())
            if not title_element:
                title_element = soup.find('h3', class_=lambda x: x and 'post-title' in str(x).lower())
            if not title_element:
                title_element = soup.find('h1')

            title = title_element.get_text(strip=True) if title_element else "no title"
        except:
            title = "no title"

        # Autor - Blogger
        try:
            author_element = soup.find('span', class_=lambda x: x and 'author' in str(x).lower())
            if not author_element:
                author_element = soup.find('a', rel='author')
            if not author_element:
                author_element = soup.find('span', itemprop='name')

            if author_element:
                author = author_element.get_text(strip=True)
                author = re.sub(r'^(Autor|By|Opublikował|Posted by):\s*', '', author, flags=re.IGNORECASE)
            else:
                # Domyślnie Jacek Głomb (to jego blog)
                author = "Jacek Głomb"
        except:
            author = "Jacek Głomb"

        # Treść artykułu - Blogger
        try:
            # Blogger używa różnych klas
            article_body = soup.find('div', class_=lambda x: x and 'post-body' in str(x).lower())
            if not article_body:
                article_body = soup.find('div', class_=lambda x: x and 'entry-content' in str(x).lower())
            if not article_body:
                article_body = soup.find('article')

            if article_body:
                text = article_body.get_text(strip=True).replace('\n', ' ').replace('\xa0', ' ')
            else:
                text = "no text"
        except:
            text = "no text"

        # Kategorie/Labels - Blogger
        try:
            # Blogger nazywa to "labels"
            label_links = soup.find_all('a', rel='tag')
            if not label_links:
                label_links = soup.find_all('a', class_=lambda x: x and 'label' in str(x).lower())

            if label_links:
                categories = [cat.get_text(strip=True) for cat in label_links]
                category = ' | '.join(categories)
            else:
                category = "no category"
        except:
            category = "no category"

        # Linki zewnętrzne
        try:
            if article_body:
                links = [a['href'] for a in article_body.find_all('a', href=True)]
                external_links = [link for link in links if not re.search(r'blogspot\.com|blogger\.com', link)]
                external_links = ' | '.join(external_links) if external_links else None
            else:
                external_links = None
        except (AttributeError, KeyError, IndexError):
            external_links = None

        # Zdjęcia
        try:
            images = []

            # 1. Zdjęcia w treści
            if article_body:
                content_images = [img['src'] for img in article_body.find_all('img', src=True) if img.get('src')]
                for img_src in content_images:
                    if img_src not in images:
                        images.append(img_src)

            # 2. Inne miejsca
            for container_class in ['post-header', 'entry-header']:
                header = soup.find('div', class_=container_class)
                if header:
                    header_images = [img['src'] for img in header.find_all('img', src=True) if img.get('src')]
                    for img_src in header_images:
                        if img_src not in images:
                            images.append(img_src)

            has_images = len(images) > 0
            photos_links = ' | '.join(images) if images else None
        except (AttributeError, KeyError, IndexError):
            has_images = False
            photos_links = None

        # Filmy (iframe)
        try:
            if article_body:
                iframes = [iframe['src'] for iframe in article_body.find_all('iframe', src=True)]
                has_videos = len(iframes) > 0
            else:
                has_videos = False
        except:
            has_videos = False

        dictionary_of_article = {
            "Link": article_link,
            "Data publikacji": date_of_publication,
            "Tytuł artykułu": title.replace('\xa0', ' '),
            "Tekst artykułu": text,
            "Autor": author,
            "Kategoria": category,
            "Linki zewnętrzne": external_links,
            "Zdjęcia/Grafika": has_images,
            "Filmy": has_videos,
            "Linki do zdjęć": photos_links
        }

        all_results.append(dictionary_of_article)

    except AttributeError as e:
        errors.append(article_link)
        print(f"Błąd dla {article_link}: {e}")
    except Exception as e:
        errors.append(article_link)
        print(f"Nieoczekiwany błąd dla {article_link}: {e}")


#%% main execution

if __name__ == "__main__":
    # Wczytaj linki z pliku
    try:
        with open('glomb_linki.txt', 'r', encoding='utf-8') as f:
            article_links = [line.strip() for line in f if line.strip()]
        print(f"Wczytano {len(article_links)} linków z pliku")
    except FileNotFoundError:
        print("Nie znaleziono pliku glomb_linki.txt")
        print("Użyj najpierw get_glomb_links.py aby pobrać linki!")
        article_links = []

    if not article_links:
        print("Brak linków do przetworzenia!")
        exit(1)

    all_results = []
    errors = []

    print("\n" + "="*60)
    print("Rozpoczynam scraping artykułów z jacekglomb.blogspot.com")
    print("="*60 + "\n")

    # Scraping z progress barem
    with ThreadPoolExecutor(max_workers=5) as executor:
        list(tqdm(executor.map(dictionary_of_article, article_links), total=len(article_links)))

    # Zapisywanie wyników
    timestamp = datetime.today().date()

    # JSON
    with open(f'glomb_{timestamp}.json', 'w', encoding='utf-8') as f:
        json.dump(all_results, f, ensure_ascii=False, indent=2, default=str)

    # Excel
    df = pd.DataFrame(all_results)
    with pd.ExcelWriter(f"glomb_{timestamp}.xlsx",
                       engine='xlsxwriter',
                       engine_kwargs={'options': {'strings_to_urls': False}}) as writer:
        df.to_excel(writer, 'Posts', index=False)

    # Raport
    print(f"\n{'='*60}")
    print(f"Scraping zakończony!")
    print(f"Przetworzono artykułów: {len(all_results)}")
    print(f"Błędów: {len(errors)}")
    if errors:
        print(f"\nLinki z błędami (pierwsze 10):")
        for error_link in errors[:10]:
            print(f"  - {error_link}")
        if len(errors) > 10:
            print(f"  ... i {len(errors) - 10} więcej")
    print(f"\nPliki wyjściowe:")
    print(f"  - glomb_{timestamp}.json")
    print(f"  - glomb_{timestamp}.xlsx")
    print(f"{'='*60}\n")

Wczytano 149 linków z pliku

Rozpoczynam scraping artykułów z jacekglomb.blogspot.com



100%|██████████| 149/149 [00:24<00:00,  6.07it/s]
/tmp/ipython-input-3838759957.py:260: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  df.to_excel(writer, 'Posts', index=False)



Scraping zakończony!
Przetworzono artykułów: 149
Błędów: 0

Pliki wyjściowe:
  - glomb_2026-01-12.json
  - glomb_2026-01-12.xlsx



In [ ]:
df.head()